In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout,Flatten,Dense
from keras.models import Sequential
from keras import applications

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
img_width, img_height = 150, 150

In [24]:
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [25]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)
    model=applications.VGG16(include_top=False,weights='imagenet')
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),bottleneck_features_train)
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),bottleneck_features_validation)

In [26]:
def train_top_model():
    train_data=np.load(open('bottleneck_features_train.npy','rb'))
    
    train_labels=np.array([0]*int(nb_train_samples/2)+[1]*int(nb_train_samples/2))
    validation_data=np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels=np.array([0]*(int(nb_validation_samples/2))+[1]*int((nb_validation_samples/2)))
    model=Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(train_data,train_labels,epochs=epochs,batch_size=batch_size,
             validation_data=(validation_data,validation_labels))
    model.save_weights(top_model_weights_path)

In [27]:
save_bottlebeck_features()
train_top_model()

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.6226 - acc: 0.7725 - val_loss: 0.3596 - val_acc: 0.8400
Epoch 2/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3497 - acc: 0.8595 - val_loss: 0.2895 - val_acc: 0.8912
Epoch 3/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3005 - acc: 0.8860 - val_loss: 0.2981 - val_acc: 0.8800
Epoch 4/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2839 - acc: 0.8895 - val_loss: 0.3239 - val_acc: 0.8888
Epoch 5/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2394 - acc: 0.9145 - val_loss: 0.3602 - val_acc: 0.8825
Epoch 6/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2022 - acc: 0.9130 - val_loss: 0.3929 - val_acc: 0.8738
Epoch 7/50
2000/2000 [==============================] - 3s 2ms/ste